In [1]:
import pygame
import numpy as np
from point import Point

pygame 2.5.2 (SDL 2.28.3, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def count_clusters(points):
    clusters = set()
    for p in points:
        if p.cluster is not None:
            clusters.add(p.cluster)
    return len(clusters)

In [3]:
def redraw(points):
    for p in points:
        if p.cluster is not None:
            color = [100 * (p.cluster + 1) % 256,
                     200 * (p.cluster + 1) % 256,
                     300 * (p.cluster + 1) % 256]
            pygame.draw.circle(screen, color=color, center=[p.x, p.y], radius=2)
        else:
            pygame.draw.circle(screen, color="#000000", center=[p.x, p.y], radius=2)
    pygame.display.set_caption(f'Cluster count: {count_clusters(points)}')
    pygame.display.update()

In [4]:
def dist(a, b):
    return np.sqrt((a.x - b.x) ** 2 + (a.y - b.y) ** 2)

In [5]:
def clear_clusters(points):
    for p in points:
        p.cluster = None
    return points

In [6]:
def find_neighbours(origin, points, epsilon=20):
    neighbours = set()
    for b in points:
        if dist(origin, b) <= epsilon:
            neighbours.add(b)
    return neighbours

In [7]:
def dbscan(points, min_neighbours_count=2, epsilon=20):
    clusters_count = 0
    checked_points = []
    searchable_points = clear_clusters(points).copy()
    green_points = []
    yellow_points = []
    while len(searchable_points) > 0:
        point = searchable_points[0]
        searchable_points.remove(point)
        checked_points.append(point)
        neighbours = find_neighbours(point, searchable_points, epsilon)
        if len(neighbours) == 0:
            continue
        if len(neighbours) >= min_neighbours_count:
            point.cluster = clusters_count
            clusters_count += 1
            green_points.append(point)
            checked_neighbours = set()
            checked_neighbours.add(point)
            while len(neighbours) > 0:
                n = neighbours.pop()
                checked_neighbours.add(n)
                if searchable_points.__contains__(n):
                    checked_points.append(n)
                    searchable_points.remove(n)
                n.cluster = point.cluster
                new_neighbours = find_neighbours(n, searchable_points, epsilon)
                neighbours |= new_neighbours.difference(checked_neighbours)
                if len(new_neighbours) >= min_neighbours_count:
                    green_points.append(n)
        else:
            yellow_points.append(point)

    for a in yellow_points:
        for b in green_points:
            if dist(a, b) <= epsilon:
                a.cluster = b.cluster
                break
    redraw(checked_points)
    return checked_points

In [ ]:
pygame.init()
screen = pygame.display.set_mode((600, 400))
flag = True
screen.fill(color="#ffffff")
pygame.display.update()
drawing = False
points = []
while flag:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            flag = False
            break
        if event.type == pygame.KEYDOWN:
            if event.key == 13:
                points = dbscan(points)
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                drawing = True
        if event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                drawing = False
        if drawing:
            coords = event.pos
            points.append(Point(coords[0], coords[1]))
            pygame.draw.circle(screen, color="#000000", center=coords, radius=2)
            pygame.display.update()

247
337
338
371
372
373
374
